# Amazon Personalize Campaign 배포

이 노트북은 기본 커널을 conda_python3를 사용합니다.
<br>
처음부터 순차적으로 끝까지 실행하여 Personalize Campaign이 배포 합니다.

## 0. 환경 셋업

필요시 boto3 sdk 업그레이드를 실행 합니다.


### boto3 업그레이드

In [ ]:
# !pip install boto3 --upgrade

## 1. 데이터 준비

아래의 Git에서 인공적으로 생성된 데이터 세트를 사용 함.
- 생선데 데이터 세트를 tar 로 압축하여 현재의 git 에 저장 했습니다. 이를 압축해제 해서 사용합니다.

* Retail Demo Store
    * https://github.com/aws-samples/retail-demo-store

In [ ]:
import tarfile

tf = tarfile.open("../data/RetailDemoDataSet.tar")
tf.extractall("../data")

In [ ]:
import pandas as pd

items = pd.read_csv('../data/items.csv')
users = pd.read_csv('../data/users.csv')
its = pd.read_csv('../data/interactions.csv')

## 2. 데이터 전처리

In [ ]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt

### Items 컬럼 조정

In [ ]:
items.columns

In [ ]:
def item_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'ITEM_ID',
                              'name' : 'NAME',
                              'category' :'CATEGORY_L1',
                              'style' : 'STYLE',
                              'description' : 'PRODUCT_DESCRIPTION',
                              'price' : 'PRICE',
                             })
    return ldf


item_cols = ['id', 'name', 'category', 'style', 'description','price']
items_df = item_data_selection(items, item_cols)    

items_df.head(3)

### Users 컬럼 조정

In [ ]:
users.columns

In [ ]:
def user_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'USER_ID',
                              'username' : 'USER_NAME',
                              'age' :'AGE',
                              'gender' : 'GENDER',                              
                             })
    return ldf

user_cols = ['id', 'username', 'age', 'gender']

users_df = user_data_selection(users, user_cols)    
users_df.head(3)

### Users 데이터 타입 변경

In [ ]:
users_df.info()

In [ ]:
def change_data_type(df, col, target_type):
    ldf = df.copy()
    ldf[col] = ldf[col].astype(target_type)
    
    return ldf

users_df = change_data_type(users_df, col='USER_ID', target_type='object')
users_df.info()

### Interactions 컬럼 조정

In [ ]:
its.columns

In [ ]:
def interactions_data_selection(df, cols):
    ldf = df[cols]
    ldf = ldf.rename(columns={'id':'USER_ID',
                              'username' : 'USER_NAME',
                              'age' :'AGE',
                              'gender' : 'GENDER',                              
                             })
    return ldf

interactions_cols = ['ITEM_ID', 'USER_ID', 'EVENT_TYPE', 'TIMESTAMP']

full_interactions_df = interactions_data_selection(its, interactions_cols)    
full_interactions_df.head(3)

### Interaction의 EVENT_TYPE 선택

여기서 EVENT_TYPE이 ProductViewd, OrderCompleted 만 선택하고 이름을 각각 View, Purchase 로 변경

In [ ]:
full_interactions_df.EVENT_TYPE.value_counts()

In [ ]:
def filter_interactions_data(df, kinds_event_type):
    ldf = df[df['EVENT_TYPE'].isin(kinds_event_type)]
    ldf['EVENT_TYPE'] = ldf['EVENT_TYPE'].replace(['ProductViewed'],'View')    
    ldf['EVENT_TYPE'] = ldf['EVENT_TYPE'].replace(['OrderCompleted'],'Purchase')        
    
    return ldf

select_event_types = ['ProductViewed','OrderCompleted']
interactions_df = filter_interactions_data(full_interactions_df, select_event_types)
interactions_df

### Interaction 데이터 타입 변경

In [ ]:
interactions_df.info()

In [ ]:
interactions_df = change_data_type(interactions_df, col='USER_ID', target_type='object')
interactions_df.info()